# Project 3: Quantum Dynamics
##    Eduardo Villaseñor - 4624548
##    Eoin Horgan - 4582322



# Introducction

In this project we use numerical methods to simulate quantum systems by means of the time-dependent Schodinguer equation:
$$-i \frac{\partial\psi}{\partial t} = H \psi $$

To use nuemerical methods to use this equation two steps are required. Discretizing the Hamiltonian into a matrix by using a discretization of the space into a mesh. And the handling the time evolution (time derivative in the Schoringuer equation) by use of the Hamiltonian.

## Discretization of the Hamiltonian

The Hamiltonian is a operator that contains all the imformation about the dynamics of a quantum system
$$ H = - \frac{\hbar}{2m}\nabla^2  + \hat{V} $$

In order to model the Hamiltonian into a matrix we need to introducre a second order finite difference discretization of the problem into an uniform mesh of width $h$, to obtain a matrix of dimensions $h^{dim} \times h^{dim}$ where $dim$ equals to the dimension of the problem. Using the Taylor expansion each the second order derivatives in the
Hamiltonian can be aproximated as follows

$$ \frac{u(x+h,y) + u(x-h,y) - 2u(x,y)}{h^2}   -\frac{\partial^2 u}{\partial x^2} = 2 \frac{h^2}{4!} \frac{\partial^4 u}{\partial x^4} + \cdots $$

Where the error is of the order of $h$. Thus by we can see that by taking finer mesh we sacrifice computing time for precision in our model. Is important to note that the matrix obtained regarless of the dimension of our problem is a sparse matrix, thus by using *scipy.sparse* tools we can handle this matrices with much ease.


## Time evolution
The time evolution of the wave function is found using the Crank-Nicolson method:
$$\psi^{n+1} = \frac{1-i\Delta t H_D/2}{1+i\Delta t H_D/2} \psi^n $$

Where the in this case the time is discretized into $\Delta t$ steps. 


## Simulations

The simulation relies on the files quantum_plots.py, simulation.py and matrix.py, which should be stored in a local directory so that it can access them. If you want to get a better idea of how the simulation works then please examine those files.

In [ ]:
import quantum_plots as qplots
import numpy as np
import simulation as sm
from IPython.display import HTML
%load_ext autoreload
%autoreload 2

### Potential Well

In [ ]:
# Define the system parameters and domain
dim = 1
numberPoints = 256
dt = .001
dirichletBC = False
startPoint = 0
domainLength = 15

sign = -1
if dirichletBC:
    sign = 1


def potentialWell(x):
    '''A top hat potential function.'''
    mag = 200
    domain = [6, 10]
    if domain[0] < x < domain[1]:
        return mag
    else:
        return 0


potentialFunc = np.vectorize(potentialWell)


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=potentialFunc,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="plane", energy=500, center=2)

# System evolution and Animation
ani = qplots.animation1D(sim, psi='real', V=potentialWell, time=400)
HTML(ani.to_html5_video())

The wave gives the behavour we expect, and at any time we can check to see if the probabilities sum to 1.

In [ ]:
P = np.sum(sim.normPsi())
if abs(P-1) < .001:
    print(True)
else:
    print(False)

The is implemented as the .consistencyCheck() method. It is also possible to view the probability over time, to verify that it remains at 1.

In [ ]:
P = sim.probability(20)

qplots.probabilityGraph(P)

### Gaussian

In [ ]:
# Define the system parameters and domain
dim = 1
numberPoints = 256
dt = .001
dirichletBC = False
startPoint = 0
domainLength = 15


def gaussian(x):
    '''A 1D Gaussian potential function'''
    mag = 200
    center = 4
    std = 1
    V = mag * np.exp(-(x-center)**2/(2*std**2))
    return V


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=gaussian,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="plane", energy=500, center=2)

# System evolution and Animation
ani = qplots.animation1D(sim, psi='real', V=gaussian, time=400)
HTML(ani.to_html5_video())

In [ ]:
sim.consistencyCheck()

## 2D Simulations:
### Double Slit interference

In [ ]:
# Define the system parameters and domain
dim = 2
numberPoints = 100
dt = .001
dirichletBC = False
startPoint = [0, 0]
domainLength = 2


def doubleSlit(x, y):
    '''A potential wall with two identical apertures'''
    sS = .3   # Slit separation
    sW = .1   # Half slit width
    spX = .5  # X coordinate start
    spY = 1   # Y coordinate of center of slits
    if x > spX and x < spX+.05 and (y < spY-sS/2. or y > spY+sS/2.):
        return 50000
    if x > spX and x < spX+.05 and (y > spY-sS/2.+sW and y < spY+sS/2.-sW):
        return 50000
    else:
        return 0


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=doubleSlit,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="plane", energy=500, center=.1, width=.1)

# System evolution and Animation
ani = qplots.animation2D(sim, psi="norm",
                     potentialFunc='none', time=200, save=False)
HTML(ani.to_html5_video())

In [ ]:
sim.consistencyCheck()

Note: If you would like to see the potential as well, then run the local files instead, there it is possible to view both overlaid on each other.

### Rutherford Dispersion

In [ ]:
# Define the system parameters and domain
dim = 2
numberPoints = 100
dt = .001
dirichletBC = False
startPoint = [0, 0]
domainLength = 2

sign = -1
if dirichletBC:
    sign = 1
allPoints = numberPoints + sign


def dispersion(x, y):
    '''1/r^2 Dispersive force around a defined center '''
    center = [0.8, 1.]
    r = np.linalg.norm([x - center[0], y - center[1]])
    return 10./r**2


def dispersionVis(x, y):
    '''An exaggereted potential function to make it more visisble'''
    center = [0.8, 1.]
    r = np.linalg.norm([x - center[0], y - center[1]])
    return 10./((r + 1)**2)


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=dispersion,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
sim.setPsiPulse(pulse="plane", energy=500, vel=1, center=.1, width=.1)

# System evolution and Animation
ani = qplots.animation2D(sim, psi="norm",
                     potentialFunc='none', save=False, time=150)
HTML(ani.to_html5_video())

In [ ]:
sim.consistencyCheck()

### Quantum Billiards

In [ ]:
# Define the system parameters and domain
dim = 2
numberPoints = 100
dt = .001
dirichletBC = False
startPoint = [0, 0]
domainLength = 1


def dispersion(x, y):
    center = [0.5, 0.5]
    r = np.linalg.norm([x - center[0], y - center[1]])
    return 15./r**2


def dispersionVis(x, y):
    center = [0.5, 0.5]
    r = np.linalg.norm([x - center[0], y - center[1]])
    if r < .2:
        return 5000
    else:
        return 0


# Create the simulation for the system
sim = sm.Simulation(dim=dim, potentialFunc=dispersionVis,
                    dirichletBC=dirichletBC, numberPoints=numberPoints,
                    startPoint=startPoint, domainLength=domainLength,
                    dt=dt)

# Create the initial wave function
# Psi = np.zeros((numberPoints + sign)**2)
#location = [np.random.choice(x, size = numPulses),
#            np.random.choice(y.flatten(), size = numPulses)]
# location = np.array([np.random.choice(x, size = numPulses),
            # np.random.choice(y.flatten(), size = numPulses)])
# location = np.swapaxes(location, 0, 1)
vel = np.random.uniform(low=-1, high=1, size=(2, 1))

# for i in range(numPulses):
#     sim.setPsiPulse(energy=500, center=location[i], vel_x=vel[0, i],
#                     vel_y=vel[1, i], width=.1)
#     Psi = Psi + sim.psi

# sim.psi = Psi
sim.setPsiPulse(pulse="circular", energy=500, center=[.2, .5], vel=[.4, .8],
                width=.1)

for i in range(30):
    sim.evolve()

# System evolution and Animation

ani = qplots.animation2D(sim, psi="norm", potentialFunc='none', save=False,
                         time=150)
HTML(ani.to_html5_video())

In [ ]:
sim.consistencyCheck()